# Email Extraction


Let's evaluate an LLM on its ability to extract structured information from email texts.

In [1]:
%pip install -U langchain langchain_benchmarks openai rapidfuzz

  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.4/289.4 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: rapidfuzz
    Found existing installation: rapidfuzz 3.0.0
    Uninstalling rapidfuzz-3.0.0:
      Successfully uninstalled rapidfuzz-3.0.0
  Attempting uninstall: openai
    Found exist

In [2]:
import os

# Get your API key from https://smith.langchain.com/settings
os.environ["LANGCHAIN_API_KEY"] = "sk-..."
os.environ["OPENAI_API_KEY"] = "sk-"

In [4]:
from langchain_benchmarks import clone_public_dataset, registry

For this code to work, please configure LangSmith environment variables with your credentials.

In [5]:
task = registry["Email Extraction"]
task

Name,Email Extraction
Type,ExtractionTask
Dataset ID,a1742786-bde5-4f51-a1d8-e148e5251ddb
Description,"A dataset of 42 real emails deduped from a spam folder, with semantic HTML tags removed, as well as a script for initial extraction and formatting of other emails from an arbitrary .mbox file like the one exported by Gmail. Some additional cleanup of the data was done by hand after the initial pass. See https://github.com/jacoblee93/oss-model-extraction-evals."


In [6]:
print(task.description)

A dataset of 42 real emails deduped from a spam folder, with semantic HTML tags removed, as well as a script for initial extraction and formatting of other emails from an arbitrary .mbox file like the one exported by Gmail.

Some additional cleanup of the data was done by hand after the initial pass.

See https://github.com/jacoblee93/oss-model-extraction-evals.
    


Clone the dataset associated with this task

In [4]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Email Extraction already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/309a2fce-ce68-43aa-befb-67f94d0c3570.


#### Schema

Each extraction task has an expected output schema defined in a Pydantic BaseModel object, which we can use to
get a JSON schema object.

In [7]:
import pprint

pprint.pprint(task.schema.schema())

{'definitions': {'ToneEnum': {'description': 'The tone of the email.',
                              'enum': ['positive', 'negative'],
                              'title': 'ToneEnum',
                              'type': 'string'}},
 'description': 'Relevant information about an email.',
 'properties': {'action_items': {'description': 'A list of action items '
                                                'requested by the email',
                                 'items': {'type': 'string'},
                                 'title': 'Action Items',
                                 'type': 'array'},
                'sender': {'description': "The sender's name, if available",
                           'title': 'Sender',
                           'type': 'string'},
                'sender_address': {'description': "The sender's address, if "
                                                  'available',
                                   'title': 'Sender Address',
                 

## Define an extraction chain

Let's build the extraction chain that we can use to get structured information from the emails.

In [9]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0).bind_functions(
    functions=[task.schema],
    function_call=task.schema.schema()["title"],
)

output_parser = JsonOutputFunctionsParser()
extraction_chain = task.instructions | llm | output_parser | (lambda x: {"output": x})

In [10]:
extraction_chain.invoke(
    {
        "input": "Hello Dear MR. I want you to send me gold to get rich."
        " First buy an envelope. Then open it and put some gold inside. "
        "Then close it and finally mail it to my address at 12345 My Gold Way."
        " You can call me any time at 000-1212-1111."
    }
)

{'output': {'sender': 'Unknown',
  'sender_phone_number': '000-1212-1111',
  'sender_address': '12345 My Gold Way',
  'action_items': ['Buy an envelope',
   'Put gold inside',
   'Close the envelope',
   "Mail it to sender's address"],
  'topic': 'Request to send gold',
  'tone': 'positive'}}

In [45]:
print (task.instructions.messages[1].prompt.template)

What can you tell me about the following email? Make sure to extract the question in the correct format. Here is the email:
 ```
{input}
```


In [36]:
task.schema.schema()

{'title': 'Email',
 'description': 'Relevant information about an email.',
 'type': 'object',
 'properties': {'sender': {'title': 'Sender',
   'description': "The sender's name, if available",
   'type': 'string'},
  'sender_phone_number': {'title': 'Sender Phone Number',
   'description': "The sender's phone number, if available",
   'type': 'string'},
  'sender_address': {'title': 'Sender Address',
   'description': "The sender's address, if available",
   'type': 'string'},
  'action_items': {'title': 'Action Items',
   'description': 'A list of action items requested by the email',
   'type': 'array',
   'items': {'type': 'string'}},
  'topic': {'title': 'Topic',
   'description': 'High level description of what the email is about',
   'type': 'string'},
  'tone': {'description': 'The tone of the email.',
   'allOf': [{'$ref': '#/definitions/ToneEnum'}]}},
 'required': ['action_items', 'topic', 'tone'],
 'definitions': {'ToneEnum': {'title': 'ToneEnum',
   'description': 'The tone 

Now it's time to measure our chain's effectiveness!

## Evaluate

Let's evaluate the chain now.

In [8]:
from langsmith.client import Client

from langchain_benchmarks.extraction import get_eval_config

In [9]:
client = Client()

In [10]:
eval_llm = ChatOpenAI(model="gpt-4", model_kwargs={"seed": 42})
eval_config = get_eval_config(eval_llm)

In [11]:
test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=extraction_chain,
    evaluation=eval_config,
    verbose=True,
    project_metadata={
        "arch": "openai-functions",
    },
)

View the evaluation results for project 'monthly-look-12' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/177d564f-516d-4b65-bae0-37154b529470?eval=true

View all tests for Dataset Email Extraction at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/309a2fce-ce68-43aa-befb-67f94d0c3570
[------------------------------------------------->] 42/42
 Eval quantiles:
                                             inputs.input  \
count                                                  42   
unique                                                 42   
top     ---  \n|\n\nEvery business faces its set of cu...   
freq                                                    1   
mean                                                  NaN   
std                                                   NaN   
min                                                   NaN   
25%                                                   NaN   
50%                       

## Compare to another LLM

Let's compare to an Anthropic LLM.

In [12]:
from langchain.chat_models import ChatAnthropic
from langchain.output_parsers.xml import XMLOutputParser
from langchain.prompts import ChatPromptTemplate

# This is the schema the model will populate
xsd = """<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema" elementFormDefault="qualified">
    <xs:element name="Email">
        <xs:complexType>
            <xs:sequence>
                <xs:element name="sender" type="xs:string" minOccurs="0"/>
                <xs:element name="sender_phone_number" type="xs:string" minOccurs="0"/>
                <xs:element name="sender_address" type="xs:string" minOccurs="0"/>
                <xs:element name="action_items" type="ActionItemsType" minOccurs="1"/>
                <xs:element name="topic" type="xs:string" minOccurs="1"/>
                <xs:element name="tone" type="ToneEnumType" minOccurs="1"/>
            </xs:sequence>
        </xs:complexType>
    </xs:element>
    <xs:complexType name="ActionItemsType">
        <xs:sequence>
            <xs:element name="item" type="xs:string" minOccurs="0" maxOccurs="unbounded"/>
        </xs:sequence>
    </xs:complexType>
    <xs:simpleType name="ToneEnumType">
        <xs:restriction base="xs:string">
            <xs:enumeration value="positive"/>
            <xs:enumeration value="negative"/>
        </xs:restriction>
    </xs:simpleType>

</xs:schema>"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a data extraction bot. Always respond "
            "only with XML of the following schema:\n{xsd}",
        ),
        (
            "user",
            "Extract Email from the folowing Document:\n"
            "<Document>\n{input}\n</Document>\n"
            "RESPOND ONLY IN XML THEN STOP.",
        ),
    ]
).partial(xsd=xsd)

claude = ChatAnthropic(model="claude-2", temperature=1)


def convert_parsed_email(email_dict: dict) -> dict:
    """Conver the XML-parsed dictionary to a flattened dict."""
    if "Email" not in email_dict:
        return email_dict
    # Flatten the tags
    result = {k: v for item in email_dict["Email"] for k, v in item.items()}
    result["action_items"] = [
        item["item"] for item in (result.get("action_items") or [])
    ]
    return {"output": result}


claude_extraction_chain = prompt | claude | XMLOutputParser() | convert_parsed_email

In [13]:
result = claude_extraction_chain.invoke(
    {
        "input": "Hello Dear MR. I want you to send me gold to get rich."
        " First buy an envelope. Then open it and put some gold inside. "
        "Then close it and finally mail it to my address at 12345 My Gold Way."
        " You can call me any time at 000-1212-1111."
    }
)
result

{'output': {'sender': None,
  'sender_phone_number': '000-1212-1111',
  'sender_address': '12345 My Gold Way',
  'action_items': ['buy an envelope',
   'open it',
   'put some gold inside',
   'close it',
   'mail it to my address'],
  'topic': 'sending gold',
  'tone': 'negative'}}

In [14]:
claude_test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=claude_extraction_chain,
    evaluation=eval_config,
    verbose=True,
    project_metadata={
        "arch": "claude-xml",
    },
)

View the evaluation results for project 'frosty-moon-4' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/81d41017-bcda-450d-8991-9bf744c7ebb8?eval=true

View all tests for Dataset Email Extraction at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/309a2fce-ce68-43aa-befb-67f94d0c3570
[-------------------------------------->           ] 33/42

Chain failed for example 9a707fca-4ba7-4f7d-8912-b9fd71e9901e with inputs {'input': "---|---|---|---  \n  \nBook with Fall Sale Extras Through November 21! Savings! OBC! Visa Gift Card\n+ More  \n  \n---  \n|\n\n|  |  |  |  |  |  |  \n---  \n|  |  \n---  \n|  | SHOP THE FALL CRUISE SALE  \n---  \n|  |  \n---  \n  \n**Celebrity Cruises** Celebrity Cruises receive **Exclusive Pricing** with\nup to **$450 BONUS Savings per Stateroom** based on double\noccupancyand even more for extra guests! Enjoy **Exclusive Tips**\non 2024 sailings, up to**$2150 Onboard Credit** , and up to a **$1700 Visa\nGiftCard** on Galapagos sailings or up to a **$650 Visa Gift Card** on\nother departures. **Drinks** and **Wi-Fi** are All Included, too!   **See=\nThis Offer =E2=96=B8**  \n  \n|  |  \n---  \n  \n**Viking** Enjoy your favorite Viking voyages with up to=C2=A0 **$1200\nShipboard Credit** from Online Vacation Center when you book by Nov 21!\nPlus, select sailings get **Airfare** , **Stateroom Upgrades**

[------------------------------------------------->] 42/42
 Eval quantiles:
                                             inputs.input  \
count                                                  42   
unique                                                 42   
top     ---  \n|\n\nEvery business faces its set of cu...   
freq                                                    1   
mean                                                  NaN   
std                                                   NaN   
min                                                   NaN   
25%                                                   NaN   
50%                                                   NaN   
75%                                                   NaN   
max                                                   NaN   

                                           outputs.output  \
count                                                  41   
unique                                                 41   
top     

## Inspect

Here, we'll take a look at the underlying results a little bit.

A few things to note:

* For this run, Anthropic is doing better on average
* The correctness is low - getting the exact information right can be difficult

In [26]:
df = test_run.to_dataframe().join(claude_test_run.to_dataframe(), rsuffix="_claude")

In [32]:
df.head(5)

,inputs.input,outputs.output,reference,feedback.json_edit_distance,feedback.score_string:accuracy,error,execution_time,inputs.input_claude,outputs.output_claude,reference_claude,feedback.json_edit_distance_claude,feedback.score_string:accuracy_claude,error_claude,execution_time_claude
61c40266-b994-49a2-8768-d54704cee079,--- \n|\n\nEvery business faces its set of cu...,"{'sender': 'EMC Financial', 'sender_address': ...","{'output': {'tone': 'positive', 'topic': 'Busi...",0.562112,0.7,None,4.358837,--- \n|\n\nEvery business faces its set of cu...,"{'sender': 'Sam', 'sender_phone_number': '800....","{'output': {'tone': 'positive', 'topic': 'Busi...",0.301242,0.7,None,10.501042
2dcfadff-51dc-458c-8af0-f47a795d0c9b,Hello Jacob!\n\n \n\nHave you noticed thesurg...,"{'sender': 'Sam at EMC', 'action_items': ['Fil...","{'output': {'tone': 'positive', 'topic': 'Gree...",0.505338,0.7,None,3.946547,Hello Jacob!\n\n \n\nHave you noticed thesurg...,"{'sender': 'Sam at EMC', 'sender_phone_number'...","{'output': {'tone': 'positive', 'topic': 'Gree...",0.113879,0.7,None,8.511848
a9c481ba-9ca5-408c-8c9c-f29127a70f7b,"Hi there,\n\n | \n--- \n \nWe've updated ou...","{'sender': 'Crunchbase Team', 'action_items': ...","{'output': {'tone': 'positive', 'topic': 'Upda...",0.245283,0.9,None,3.972396,"Hi there,\n\n | \n--- \n \nWe've updated ou...","{'sender': None, 'sender_phone_number': None, ...","{'output': {'tone': 'positive', 'topic': 'Upda...",0.343434,0.7,None,9.739630
98358188-6e36-42ef-9298-83acf8d9dd12,Consider all ways to give to \nSave the Redwo...,"{'sender': 'Tim Whalen', 'sender_address': 'Sa...","{'output': {'tone': 'positive', 'topic': 'Dona...",0.280556,0.7,None,3.890567,Consider all ways to give to \nSave the Redwo...,"{'sender': None, 'sender_phone_number': None, ...","{'output': {'tone': 'positive', 'topic': 'Dona...",0.255556,0.3,None,9.640687
0f29e857-fc08-45dd-b1ea-dde1e00c4a62,Some travelers plan ahead; others prefer a bit...,"{'sender': 'Dunhill Vacations Inc.', 'sender_a...","{'output': {'tone': 'positive', 'topic': 'Trav...",0.552463,0.7,None,4.252478,Some travelers plan ahead; others prefer a bit...,"{'sender': 'Dunhill Vacations Inc.', 'sender_p...","{'output': {'tone': 'positive', 'topic': 'Trav...",0.584582,0.3,None,6.803259


In [30]:
(
    df["feedback.json_edit_distance"].mean(),
    df["feedback.json_edit_distance_claude"].mean(),
)

(0.5664337704936568, 0.382351925386955)

In [33]:
(
    df["feedback.score_string:accuracy"].mean(),
    df["feedback.score_string:accuracy_claude"].mean(),
)

(0.48571428571428565, 0.5658536585365853)

In [46]:
# Rows for which OAI > Claude by at least 30%, according to the LLM-based evaluator
oai_beats_claude = df[
    (df["feedback.score_string:accuracy"] - df["feedback.score_string:accuracy_claude"])
    >= 0.3
]

oai_beats_claude[["inputs.input", "outputs.output", "outputs.output_claude"]]

,inputs.input,outputs.output,outputs.output_claude
98358188-6e36-42ef-9298-83acf8d9dd12,Consider all ways to give to \nSave the Redwo...,"{'sender': 'Tim Whalen', 'sender_address': 'Sa...","{'sender': None, 'sender_phone_number': None, ..."
0f29e857-fc08-45dd-b1ea-dde1e00c4a62,Some travelers plan ahead; others prefer a bit...,"{'sender': 'Dunhill Vacations Inc.', 'sender_a...","{'sender': 'Dunhill Vacations Inc.', 'sender_p..."
35414bbc-4d38-41ed-876f-2a6a067e66d5,--- \n \n|\n\nWe Passed the Stop Dangerous P...,"{'sender': 'Matt Haney', 'sender_address': '10...","{'sender': 'Matt Haney', 'sender_phone_number'..."
ff1b2ed6-26a7-4501-96aa-6e3e10eadc72,--- \n|\n\n# We Provide Unique Financing Opti...,"{'sender': 'info@championadvance.com', 'sender...","{'sender': None, 'sender_phone_number': None, ..."


In [51]:
# Rows for which Claude > OAI by at least 50%, according to the LLM-based evaluator
oai_beats_claude = df[
    (df["feedback.score_string:accuracy_claude"] - df["feedback.score_string:accuracy"])
    >= 0.5
]

oai_beats_claude[["inputs.input", "outputs.output", "outputs.output_claude"]]

,inputs.input,outputs.output,outputs.output_claude
02cfdfc4-c3dc-47e6-ad44-8e437ebf2dce,---|---|---|--- \n \n| \n--- \n **Limited ...,"{'action_items': [], 'topic': 'Limited Time Up...","{'sender': 'Dunhill Vacations Inc.', 'sender_p..."
198dc232-8f98-484a-a65e-048cfb517282,"Hello Jacob,\n\n \n\nFor many small businesse...","{'sender': 'Sam at EMC', 'action_items': ['Kic...","{'sender': 'Sam at EMC', 'sender_phone_number'..."
c222957f-cc7e-46af-9cca-1270f3fa5621,"Hello Jacob,\n\n \n\nDo you know what Fortune...","{'sender': 'Sam at EMC', 'action_items': ['qua...","{'sender': 'Sam at EMC', 'sender_phone_number'..."
119ef037-8744-4eb9-93df-64458278e4f8,--- \n| | QUALIFY NOW \n--- \n \n \nHell...,"{'sender': 'Sam at EMC', 'action_items': ['Che...",{'sender': 'Sam at EMC id:2023-09-19-20:17:53:...
